In [58]:
import tetgen as tg
import numpy as np
import matplotlib.pyplot as pl
import meshplot as mp
import json
import meshio as mio

In [154]:
box_is_obstacle = False
sh = 1
sb = 0.1
n_spikes = 10

bh = 0.6
bb = 0.4
bho = -0.05
bbo = -0.1
a = 0.00005
# a = 100000

v = np.array([
    [-sb/2, 0, -sb/2],
    [-sb/2, 0, sb/2],
    [sb/2, 0, sb/2],
    [sb/2, 0, -sb/2],
    [0, sh, 0]])
f = np.array([
    [0, 1, 2],
    [0, 2, 3],
    [0, 1, 4],
    [1, 2, 4],
    [2, 3, 4],
    [3, 0, 4],
    ])

gen = tg.TetGen(v, f)
v, f = gen.tetrahedralize(switches=f"Qpqa{a}")

trans = np.array([[2*t*sb] for t in range(n_spikes)])
tx, tz = np.meshgrid(trans, trans)
trans = np.column_stack((tx.flatten(), np.zeros_like(tx.flatten()), tz.flatten()))
spike_center = np.mean(trans, axis=0)


box_v = np.array([
    [-bb+bbo, sh+bho, -bb+bbo],
    [bbo, sh+bho, -bb+bbo],
    [bbo, sh+bho, bbo],
    [-bb+bbo, sh+bho, bbo],

#4
    [-bb+bbo, sh+bho+bh, -bb+bbo],
    [bbo, sh+bho+bh, -bb+bbo],
    [bbo, sh+bho+bh, bbo],
    [-bb+bbo, sh+bho+bh, bbo]
])

box_f = np.array([
    [0, 1, 2],
    [0, 2, 3],

    [0, 1, 4],
    [1, 4, 5],

    [1, 2, 5],
    [2, 5, 6],

    [2, 3, 6],
    [3, 6, 7],

    [3, 0, 7],
    [0, 7, 4],

    [4, 5, 6],
    [4, 6, 7],
])


box_center = np.mean(box_v, axis=0)


gen = tg.TetGen(box_v, box_f)
box_v, box_f = gen.tetrahedralize(switches=f"Qpqa{a}")


In [155]:
radius =  spike_center - box_center

t = 0

def traj(t):
    point = [np.sin(2*np.pi*t+np.pi/4)/np.sqrt(2)*radius[0]*2, radius[1], np.cos(2*np.pi*t + np.pi/4)/np.sqrt(2)*radius[2]*2]
    # point[0] *= np.cos(2*np.pi*t)/2+0.5
    # point[2] *= np.cos(2*np.pi*t)/2+0.5

    point[0] *= (1 - t)
    point[2] *= (1 - t)
    point = spike_center - point
    return point

traj(0) - box_center
# np.cos(0)/2+0.5

radius, spike_center, box_center

(array([ 1.2 , -1.25,  1.2 ]),
 array([0.9, 0. , 0.9]),
 array([-0.3 ,  1.25, -0.3 ]))

In [156]:
pp = np.array([traj(2*t/40) for t in range(41)])
# pp

In [157]:
p = mp.plot(box_v, box_f, shading={"wireframe": True})
for t in trans:
    p.add_mesh(v+t, f)

p.add_points(pp, shading={"point_size": 1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.300000…

101

In [158]:
sim = {
    "common": "common.json",
    "geometry": []
}

for t in trans:
    sim["geometry"].append({
        "mesh": "spike.msh",
        "transformation": {
            "translation": [np.round(t[0]*1000)/1000, np.round(t[1]*1000)/1000, np.round(t[2]*1000)/1000]
        },
        "volume_selection": 1,
        "surface_selection": {
            "id": 1,
            "axis": -2,
            "position": 0.01
        }
    })

if box_is_obstacle:
    sim["geometry"].append({
        "mesh": "box.msh",
        "surface_selection": 2,
        "is_obstacle": True
    })
else:
    sim["geometry"].append({
        "mesh": "box.msh",
        "volume_selection": 2,
        "surface_selection": {
            "id": 2,
            "axis": 2,
            "position": sh+bho+bh-0.01
        }
    })

with open("sim.json", "w") as file:
    json.dump(sim, file)

cells = [("tetra", f)]
mesh = mio.Mesh(v, cells)
mesh.write("spike.msh", file_format="gmsh")

cells = [("tetra", box_f)]
mesh = mio.Mesh(box_v, cells)
mesh.write("box.msh", file_format="gmsh")

In [161]:
[
    f"{spike_center[0]} - sin(2*pi*t+pi/4)/sqrt(2)*{np.round(2*radius[0]*100)/100}*(1-t) - {np.round(box_center[0]*100)/100}",
    "0",
    f"{spike_center[0]} - cos(2*pi*t+pi/4)/sqrt(2)*{np.round(2*radius[0]*100)/100}*(1-t) - {np.round(box_center[0]*100)/100}"
]

['0.9 - sin(2*pi*t+pi/4)/sqrt(2)*2.4*(1-t) - -0.3',
 '0',
 '0.9 - cos(2*pi*t+pi/4)/sqrt(2)*2.4*(1-t) - -0.3']